In [38]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import itertools
import numpy as np

In [39]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [40]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

### Dataset and Dataloader:

In [41]:
from torch.utils.data import TensorDataset

# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [42]:
from torch.utils.data import DataLoader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [43]:
for xd, yd in train_dl:
    print('Batch:')
    print(xd, yd)

Batch:
tensor([[ 74.,  66.,  43.],
        [ 73.,  66.,  44.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 92.,  87.,  64.]]) tensor([[ 57.,  69.],
        [ 57.,  69.],
        [118., 134.],
        [ 20.,  38.],
        [ 82., 100.]])
Batch:
tensor([[ 88., 134.,  59.],
        [ 73.,  67.,  43.],
        [101.,  44.,  37.],
        [ 91.,  87.,  65.],
        [ 68.,  97.,  70.]]) tensor([[118., 132.],
        [ 56.,  70.],
        [ 21.,  38.],
        [ 80., 102.],
        [102., 120.]])
Batch:
tensor([[ 87., 134.,  58.],
        [ 68.,  96.,  71.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]]) tensor([[119., 133.],
        [104., 118.],
        [103., 119.],
        [ 22.,  37.],
        [ 81., 101.]])


### nn.Linear

In [44]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1258, -0.5694, -0.3067],
        [-0.3332,  0.2669,  0.4635]], requires_grad=True)
Parameter containing:
tensor([0.0436, 0.2218], requires_grad=True)


In [45]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1258, -0.5694, -0.3067],
         [-0.3332,  0.2669,  0.4635]], requires_grad=True),
 Parameter containing:
 tensor([0.0436, 0.2218], requires_grad=True)]

In [46]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -60.4768,   13.7104],
        [ -81.1393,   23.0510],
        [-104.9862,   33.8820],
        [ -48.6201,   -5.1402],
        [ -84.7670,   35.2982],
        [ -60.0333,   13.1103],
        [ -80.8767,   23.2475],
        [-105.4188,   34.0122],
        [ -49.0637,   -4.5401],
        [ -84.9479,   36.0949],
        [ -60.2142,   13.9070],
        [ -80.6958,   22.4508],
        [-105.2488,   33.6854],
        [ -48.4392,   -5.9369],
        [ -85.2105,   35.8984]], grad_fn=<AddmmBackward0>)

### Lost Function:

In [47]:
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

In [48]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(15785.5420, grad_fn=<MseLossBackward0>)


### Optimizer

In [49]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [50]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [51]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 453.6801
Epoch [20/100], Loss: 114.6176
Epoch [30/100], Loss: 288.3987
Epoch [40/100], Loss: 111.1286
Epoch [50/100], Loss: 34.9956
Epoch [60/100], Loss: 43.5906
Epoch [70/100], Loss: 26.8819
Epoch [80/100], Loss: 26.2571
Epoch [90/100], Loss: 35.6118
Epoch [100/100], Loss: 23.5744


In [52]:
preds = model(inputs)
preds

tensor([[ 58.5424,  70.9140],
        [ 80.6782,  99.8622],
        [118.4472, 133.4451],
        [ 29.9908,  40.4917],
        [ 94.1803, 115.7752],
        [ 57.4884,  69.8823],
        [ 80.2003,  99.8114],
        [118.6201, 133.9822],
        [ 31.0448,  41.5233],
        [ 94.7564, 116.7560],
        [ 58.0646,  70.8631],
        [ 79.6242,  98.8306],
        [118.9250, 133.4960],
        [ 29.4147,  39.5109],
        [ 95.2343, 116.8068]], grad_fn=<AddmmBackward0>)

In [53]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])